In [1]:
from Dynamics import *

In [2]:
adj,mean_patient = LoadMatData()

In [3]:
doctors = {}
doctors["originalID"] = np.arange(50).squeeze()

In [5]:
remove_doctors(doctors["originalID"], 2)

array([29, 14])